# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [93]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [94]:
city_weather_df = pd.read_csv("..\WeatherPy\output_data\cities.csv",index_col="City_ID")

city_weather_df = city_weather_df.dropna()
city_weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
City_ID,,,,,,,,,
0,albany,71,US,1593315320,91,42.60,-73.97,66.99,2.71
1,georgetown,20,MY,1593315344,74,5.41,100.34,86.00,10.29
2,emerald,75,AU,1593315345,49,-23.53,148.17,73.40,12.75
3,ngunguru,5,NZ,1593315346,66,-35.62,174.50,62.01,3.00
4,ushuaia,20,AR,1593315290,59,-54.80,-68.30,35.60,20.80
...,...,...,...,...,...,...,...,...,...
552,monrovia,94,LR,1593315716,88,6.30,-10.80,75.90,4.61
553,buchanan,95,LR,1593315717,88,5.88,-10.05,75.49,5.73
554,farafangana,15,MG,1593315717,90,-22.82,47.83,66.51,8.19


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [95]:
gmaps.configure(api_key=g_key)
locations = city_weather_df[["Lat", "Lng"]]
humidity = city_weather_df["Humidity"]


In [96]:
fig = gmaps.figure(zoom_level = 2,center = (0,0))
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [97]:
# Ideal Weather conditions
# 70 > Max_Temp < 80
# Wind_Speed < 10 mph
# Cloudiness = 0
ideal_city_weather_df = city_weather_df.loc[(city_weather_df["Max_Temp"] < 80) & (city_weather_df["Max_Temp"] > 70) &
                                           (city_weather_df["Wind_Speed"] < 10) & (city_weather_df["Cloudiness"] == 0)]
hotel_df = ideal_city_weather_df.reset_index(drop=True)
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,uruzgan,0,AF,1593315354,18,32.83,66.00,77.47,8.39
1,salsk,0,RU,1593315419,45,46.47,41.54,70.12,3.20
2,waddan,0,LY,1593315445,49,29.16,16.14,71.98,6.04
3,gonbad-e qabus,0,IR,1593315448,61,37.25,55.17,77.00,2.46
4,gat,0,IL,1593315486,86,31.61,34.76,72.00,4.54
5,tripoli,0,LY,1593315234,51,32.88,13.19,77.92,9.75
6,corinth,0,GR,1593315522,41,37.94,22.96,77.00,3.51
7,vila velha,0,BR,1593315580,83,-20.33,-40.29,72.00,3.36
8,zenzeli,0,RU,1593315610,30,45.92,47.05,74.53,9.15
9,tahta,0,EG,1593315634,51,26.77,31.50,74.73,8.63


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [98]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for i, row in hotel_df.iterrows():
    params = {
        "location": f"{row['Lat']},{row['Lng']}",  
        "type": "lodging",
        "radius": 5000,
        "keyword": "Hotel",    
        "key": g_key
    }
    
    response = requests.get(base_url, params=params).json()

    # extract results
    results = response['results']   
    try:
        hotel_df.at[i,"Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        hotel_df.at[i,"Hotel Name"] = ""
        
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel Name
0,uruzgan,0,AF,1593315354,18,32.83,66.00,77.47,8.39,
1,salsk,0,RU,1593315419,45,46.47,41.54,70.12,3.20,Sancy Hotel
2,waddan,0,LY,1593315445,49,29.16,16.14,71.98,6.04,
3,gonbad-e qabus,0,IR,1593315448,61,37.25,55.17,77.00,2.46,Hotel
4,gat,0,IL,1593315486,86,31.61,34.76,72.00,4.54,Desert Gat
5,tripoli,0,LY,1593315234,51,32.88,13.19,77.92,9.75,Radisson Blu Al Mahary Hotel
6,corinth,0,GR,1593315522,41,37.94,22.96,77.00,3.51,Hotel Mantas Seaside
7,vila velha,0,BR,1593315580,83,-20.33,-40.29,72.00,3.36,Sheraton Vitoria Hotel
8,zenzeli,0,RU,1593315610,30,45.92,47.05,74.53,9.15,
9,tahta,0,EG,1593315634,51,26.77,31.50,74.73,8.63,Taqwa Club


In [99]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [100]:




# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))